In [ ]:
# Boilerplate
import numpy as np
import pandas as pd
import pandas_text as pt

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

# Initialize the spaCy deep parser
parser = spacy.load("en_core_web_sm")

# Parse a sentence
target_text = "The luxury auto maker bot last year sold 1,214 cars in the U.S."
token_features = pt.make_tokens_and_features(target_text, parser)
token_features

In [ ]:
# Show the dependency parse of the sentence
# Note that this is a bit different from the gold-standard parse tree.
pt.render_parse_tree(token_features)

Original rule is:
```yaml
-
  condition: { /node/pos: NOUN }
  actions:
    - function: strip_phrase
 #     input : { node: /node}
      params: { excludePos: '[ "DET", "ADJ" ]' }
      output : { strippedPhrase: /strippedSpan, strippedPhraseNF: /normalForm }
  outputs: [{ view: NounPhrase, id: /node/id, span: /node/span_of, 
          head: /node, headPOS: /node/pos, headNF: /node/lemma,
          determiner: /node/children/pos=DET,
          strippedPhrase: /strippedPhrase, strippedPhraseNF: /strippedPhraseNF }]     
```
Here's the source code for the `strip_phrase` UDF that this rule depends on:
```java
	private DataObj stripPhrase(Data inputObj, Map<String, List<String>> paramSet, Trace trace) {
		if (_trace) trace.push("getStringExcludePos");
		Node node = (Node) inputObj.get("node", trace);
		
		// If node already has required output, return it
		DataObj output = (DataObj) node.stringRecursive;
		if (output != null) {
			if (_trace) trace.pop();
			return output;
		}

		// Preprocess 
		List<String> excludePosList = paramSet.get("excludePos");
		Set<String> excludePos = new HashSet<>(excludePosList);
		
		DataObj result = stripPhrase(node, excludePos, trace);

		if (_trace) trace.pop();
		return result;
	}

	private DataObj stripPhrase(Node node, Set<String> excludePos, Trace trace) {
		if (_trace) trace.push("stringRecursive for " + node.id);
		List<Span> spans = new ArrayList<>();
		List<String> lemmas = new ArrayList<>();
		
		// From left
		for (Node child: node.children) {
			if (child.id < node.id ) {
				DataObj results = stripPhrase(child, excludePos, trace);
				spans.add((Span) results.get("strippedSpan"));
				lemmas.add(results.getBareString("normalForm"));
			}
		}
		
		// From self
		if (! excludePos.contains(node.pos)) {
			spans.add(node.getNodeSpan(trace));
			lemmas.add(node.lemma);
		}

		// Combine to form results
		Span span = Span.combine(spans, trace);
		String normalForm = StringUtils.join(lemmas, " ").trim();
		DataObj results = new DataObj();
		results.put("strippedSpan", span);
		results.put("normalForm", normalForm);
		node.stringRecursive = results;

		if (_trace) trace.pop();
		return results;
	}

```

English language translation of the above:
1. Start with every token tagged `NOUN`
1. Find every child of each `NOUN` token that is to the left of the `NOUN` and is not tagged with `DET` or `ADJ`
1. Recursively repeat the previous step until a fixed point is reached.
1. Also return the set of children of the head noun tagged with DET as the "determiner".  Leave the "determiner" field blank of no such children are present.
1. For each set of children, find the smallest span that covers all children. Return that span as the "stripped" span.


In [ ]:
# Wrap a Gremlin GraphTraversal around our token features DataFrame 
g = pt.token_features_to_traversal(token_features)

# The parts of the rule that naturally translate to Gremlin we do in Gremlin.
noun_phrase_traversal = (
    g.V()
    # 1. Start with every token tagged `NOUN`
    .has("pos", "NOUN").as_("head", "headPOS", "headNF")
    # 2. Find every child of each NOUN token that is to the left of the NOUN and is not 
    #    tagged with DET or ADJ
    # 3. Recursively repeat the previous step until a fixed point is reached
    .repeat(pt.__.in_()
            .where(pt.lt("head")).by("token_span")
            .has("pos", pt.without("DET", "ADJ"))).emit().as_("child")
    # 4. Also return the set of children of the head noun tagged with DET as the 
    #   "determiner" field.  Leave the field blank if no such children are present.
    .coalesce(
        pt.__.select("head").in_().has("pos", "DET").values("token_span"),
        pt.__.constant(None)).as_("determiner")
    .select("head", "headPOS", "headNF", "child", "determiner")
        .by("token_span").by("pos").by("lemma").by("token_span").by()
    .compute()
)
# The aggregation and formatting parts of the rule are in Pandas.
# 5. For each set of children, find the smallest span that covers all children. 
#    Return that span as the "stripped" span.
noun_phrase_df = (noun_phrase_traversal
                  .toDataFrame()
                  .groupby(["head"]).aggregate({"headPOS": "first", "headNF": "first", 
                                                "determiner": "first",
                                                "child": pt.combine_agg})
                  .reset_index())
noun_phrase_df["strippedSpan"] = pt.combine_spans(noun_phrase_df["child"], noun_phrase_df["head"])
noun_phrase_df["normalForm"] = pt.lemmatize(noun_phrase_df["strippedSpan"], token_features)
noun_phrase_df

In [ ]:
print(pt.token_features_to_gremlin(token_features, include_begin_and_end=True))